In [1]:

import pandas as pd
import numpy as np
import pickle
from itertools import islice
from sklearn import preprocessing  # Two samples, with 3 dimensions.



path = './data/'

embedding_names = ['bow5', 'bow2']
embedding_names = ['bow5', 'deps']


simlexf = 'Simlex/SimLex-999.txt'

#
# embeddings: list of dicts holding the data for each embedding
# dict: {name - embedding name,
#       filename - embedding data filename
#       words - dict holding the words  and their vectors
#    
embeddings = []


for name in embedding_names:
    # words: dict, holds all words and their vectors for each embedding
    words = {}
    words_normalized = {}
    filename = name + '.words.bz2'
    embedding_df = pd.read_table(path + filename , sep=' ', header=None)
    for index, row in embedding_df.iterrows():
        words[row[0]] = row[1:]
    
    
    embedding_vectors = np.stack(list(words.values()))
    embedding_vocabulary = np.stack(list(words.keys()))
    embeddings.append({'name':name, 'filename': filename, 
                            'words': words, ' embedding_vectors':  embedding_vectors,
                       'embedding_vocabulary': embedding_vocabulary})
        




In [2]:
#
# Compute the cosine similarity for each word pair in the simlex
#


import analyze
import evaluate
import imp

evaluate = imp.reload(evaluate)
analyze = imp.reload(analyze)

    

cosine_similarities = []


path = './data/'
simlexf = 'SimLex-999/SimLex-999.txt'
simlex = pd.read_table(path + simlexf)
simlex_sim =  np.array(simlex["SimLex999"]).astype(np.float)

MEN_f = 'MEN/MEN_dataset_natural_form_full'
MEN = pd.read_table(path + MEN_f, header=None, sep=' ')
MEN_sim =  np.array(MEN.ix[:,2]).astype(np.float)

similarity_evaluation = pd.DataFrame({})



word_pairs = simlex

spearman = []
pearson = []

datasets = {'simlex': simlex_sim, 'MEN': MEN_sim}


def compute_correlations(similarities, predicted):
        
    df = pd.DataFrame({'control': similarities, 'predicted': predicted})
    return( {'pearson': df.corr(method = 'pearson')['control'][1], 
            'spearman': df.corr(method = 'spearman')['control'][1]})
    

# Calculate cosine similarity between each pair of words in the simlex

words_pairs = simlex 
dataset = 'simlex'
for embedding in embeddings:
    word_vectors = embedding['words']
    cosine_similarities = []
    cosine_similarities = evaluate.evaluate_similarity(word_pairs, word_vectors)
    embedding.update({'cosine_similarities':cosine_similarities})

    cors = compute_correlations(simlex_sim, cosine_similarities)
    pearson.append(cors['pearson'])
    spearman.append(cors['spearman'])
    
similarity_evaluation = pd.DataFrame({'embedding': embedding_names, 
                                      'spearman': spearman,
                                     'pearson': pearson})


print(similarity_evaluation)


spearman = []
pearson = []   

word_pairs = MEN

for embedding in embeddings:
    word_vectors = embedding['words']
    cosine_similarities = []
    cosine_similarities = evaluate.evaluate_similarity(word_pairs, word_vectors)
    embedding.update({'cosine_similarities':cosine_similarities})

    cors = compute_correlations(MEN_sim, cosine_similarities)
    pearson.append(cors['pearson'])
    spearman.append(cors['spearman'])
    
        
similarity_evaluation = pd.DataFrame({'embedding': embedding_names, 
                                      'spearman': spearman,
                                     'pearson': pearson})


    
print(similarity_evaluation)



    

  embedding   pearson  spearman
0      bow5  0.375601  0.367396
1      bow2  0.428459  0.414146
  embedding   pearson  spearman
0      bow5  0.708236  0.723169
1      bow2  0.677698  0.699905


In [5]:
#
# Qualitative checks
#
#
# Look into 5 most similar words
word_to_check='old'
#word_to_check='smart'

for embedding in embeddings:
    words = embedding['words']
    most_similar = closest_words(embedding['embedding_vectors'],
                                 embedding['embedding_vocabulary'], words[word_to_check], 6, exclude=[])
    
    print(most_similar)



KeyError: 'embedding_vectors'

In [ ]:
all_words = np.stack(list(words2.values()))
all_keys = np.stack(list(words2.keys()))



In [ ]:

def normalize_words(vectors):
    """Normalize embeddings matrix row-wise.
    Parameters
    ----------
      ord: normalization order. Possible values {1, 2, 'inf', '-inf'}
    """
    print(np.linalg.norm(vectors[50]))
    print(np.linalg.norm(vectors, axis=1))
    vectors = vectors.T / np.linalg.norm(vectors, axis=1)
    return vectors.T

normalized_matrix = normalize_words(embedding_vectors)


In [ ]:
import analyze
import evaluate
import imp

analyze = imp.reload(analyze)

words = embeddings[0]['words']
embedding_vectors = np.stack(list(words.values()))
embedding_vocabulary = np.stack(list(words.keys()))


    
    

## ANALOGY TASK

In [4]:

def closest_words(embedding_vectors,embedding_vocabulary, word, k, exclude=[]):
    #
    # Obtain the k most similar words based on cosine similarity
    #
    D = pairwise_distances(embedding_vectors, word.reshape(1, -1), metric='cosine')
    possible_answers=[]
    kwords=k
    if exclude:
        kwords=k+len(exclude)
    for id in D.argsort(axis=0).flatten()[0:kwords]:
        
        # Exclude words in query
        if embedding_vocabulary[id] not in (exclude):
            possible_answers.append(embedding_vocabulary[id])
    return(possible_answers)

In [ ]:
# Analogy Task


from sklearn.metrics import pairwise_distances

# Compute offset
analogy_file = 'questions-words.txt'
analogy_words = pd.read_table(path + analogy_file , sep=' ', header=1)
analogy_words.head()


for embedding in embeddings:
    if embedding['name'] == 'deps':
        words = embeddings[0]['words']
        embedding_vectors = np.stack(list(words.values()))
        embedding_vocabulary = np.stack(list(words.keys()))

        # warning here I only used the 9000 first words!
        # also breaks on first one
        indexes_found = []
        answers = []
        for index, row in analogy_words.iterrows():

                if row[0] in words.keys() and row[1] in words.keys() and row[2] in words.keys():

                    a = words[row[0]]
                    astar = words[row[1]]
                    b = words[row[2]]

                    # Compute offset a* - a, add to b
                    bstar = b.T + (astar.T - a.T)            
                    cosine_sims = []
                    cosine_sims_keys = []
                    # Now that we computed  bstar find the closest vector to it
                    # compute cosine similarity with all vectors
                    # obtain the closest 

                    exclude = [row[0], row[1], row[2]]

                    possible_answers = closest_words(embedding_vectors,embedding_vocabulary, 
                                                     bstar, 6, exclude)


                    answer = possible_answers[0]
                    indexes_found.append(index)
                    answers.append(possible_answers[1:k])

        embedding.update({'analogy_answers':possible_answers, 
                          'analogy_indexes':indexes_found})







#        with open('answers_' + embedding['name'] + '.csv','w') as f:
#           for r in answers:
#                f.write(str(r)+'\n')

                      
            

In [ ]:
# Just save the found indices
for embedding in embeddings:
    if embedding['name'] == 'deps':
        words = embeddings[0]['words']
        embedding_vectors = np.stack(list(words.values()))
        embedding_vocabulary = np.stack(list(words.keys()))

        # warning here I only used the 9000 first words!
        # also breaks on first one
        indexes_found = []
        answers = []
        for index, row in analogy_words.iterrows():
                if row[0] in words.keys() and row[1] in words.keys() and row[2] in words.keys():
                    indexes_found.append(index)
 

    with open('answers_' + embedding['name'] + 'words_found.csv','w') as f:
        for r in indexes_found:
            f.write(str(r)+'\n')

In [ ]:
with open('answers_' + embedding['name'] + 'words_found.csv','w') as f:
    for r in indexes_found:
        f.write(str(r)+'\n')


### Evaluate Answers

In [ ]:
with open('answers_' + embedding['name'] + '.csv','r') as f:
    for line in f.readlines():
        